<a href="https://colab.research.google.com/github/Kunj-13/StockSense/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW 5

Part 1:

This code measures the time taken to communicate between a parent and a child process using shared memory for different data sizes. The parent process writes random data to shared memory, and the child process reads and rewrites it. Both processes detach from shared memory afterward. Time spent during this communication is calculated and displayed for each data size (10KB, 100KB, 1MB, and 10MB).

In [ ]:
%%writefile shared_memory.c
// Include necessary libraries for shared memory, process management, etc.
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>  // For various data types
#include <sys/ipc.h>    // For interprocess communication (IPC)
#include <sys/shm.h>    // For shared memory management
#include <sys/wait.h>   // For wait functions
#include <unistd.h>     // For fork, shmat, shmdt, etc.
#include <time.h>       // For time measurement

// Define sizes for experimentation
#define SIZE_10KB 10240      // 10KB in bytes
#define SIZE_100KB 102400    // 100KB in bytes
#define SIZE_1MB 1048576     // 1MB in bytes
#define SIZE_10MB 10485760   // 10MB in bytes

// Function to fill an array with random integers
void fill_array(int *arr, int size) {
    // Iterate over the array and assign random integers
    for (int i = 0; i < size / sizeof(int); i++) {
        arr[i] = rand() % 100;  // Random integer between 0 and 99
    }
}

int main() {
    int shmid;                  // Shared memory ID
    int *shared_memory;         // Pointer to the shared memory
    pid_t pid;                  // Process ID for fork

    // Array of sizes to experiment with
    int data_sizes[] = {SIZE_10KB, SIZE_100KB, SIZE_1MB, SIZE_10MB};
    // Number of data sizes in the array
    int num_sizes = sizeof(data_sizes) / sizeof(data_sizes[0]);

    // Loop through each data size for experimentation
    for (int j = 0; j < num_sizes; j++) {
        int data_size = data_sizes[j];  // Set the current data size

        // Time measurement start
        clock_t start, end;      // Variables to store start and end times
        double time_spent;       // Variable to store the time spent

        start = clock();         // Record the start time

        // Create shared memory segment
        shmid = shmget(IPC_PRIVATE, data_size, IPC_CREAT | 0666);
        if (shmid < 0) {         // Check for errors in shared memory creation
            perror("shmget failed");
            exit(1);             // Exit if shared memory creation failed
        }

        // Fork the process to create a parent and child process
        pid = fork();

        if (pid < 0) {           // Check if fork failed
            perror("Fork failed");
            exit(1);             // Exit if fork failed
        }

        if (pid == 0) {          // Child process
            shared_memory = (int *)shmat(shmid, NULL, 0);  // Attach to shared memory
            if (shared_memory == (int *)-1) {  // Check if attachment failed
                perror("Child shmat failed");
                exit(1);         // Exit if shared memory attachment failed
            }
            // Print message from the child process
            printf("Child: Reading data from shared memory of size %d bytes...\n", data_size);
            fflush(stdout);  // Flush the output buffer to ensure message is printed

            // Write the array back to shared memory (essentially no-op in this case)
            for (int i = 0; i < data_size / sizeof(int); i++) {
                shared_memory[i] = shared_memory[i];
            }
            printf("Child: Writing data back to shared memory.\n");
            fflush(stdout);  // Flush the output buffer

            // Detach from shared memory
            shmdt(shared_memory);
            exit(0);         // Child process exits
        } else {
            // Parent process
            shared_memory = (int *)shmat(shmid, NULL, 0);  // Attach to shared memory
            if (shared_memory == (int *)-1) {  // Check if attachment failed
                perror("Parent shmat failed");
                exit(1);         // Exit if shared memory attachment failed
            }
            // Print message from the parent process
            printf("Parent: Writing data to shared memory of size %d bytes...\n", data_size);
            fflush(stdout);  // Flush the output buffer

            // Fill shared memory with random integers
            fill_array(shared_memory, data_size);

            // Wait for the child process to complete
            wait(NULL);

            // Read data from shared memory (no-op in this case)
            printf("Parent: Reading data from shared memory.\n");
            fflush(stdout);  // Flush the output buffer

            // Detach from shared memory
            shmdt(shared_memory);
            // Remove the shared memory segment
            shmctl(shmid, IPC_RMID, NULL);
        }

        // Time measurement end
        end = clock();  // Record the end time
        time_spent = (double)(end - start) / CLOCKS_PER_SEC;  // Calculate the elapsed time
        // Print the time taken for the shared memory communication
        printf("Time taken for shared memory communication of size %d bytes: %f seconds\n\n", data_size, time_spent);
        fflush(stdout);  // Flush the output buffer
    }

    return 0;  // End of the program
}


Overwriting shared_memory.c


In [ ]:
!gcc shared_memory.c -o shared_memory -lrt
!./shared_memory

Parent: Writing data to shared memory of size 10240 bytes...
Child: Reading data from shared memory of size 10240 bytes...
Child: Writing data back to shared memory.
Parent: Reading data from shared memory.
Time taken for shared memory communication of size 10240 bytes: 0.000326 seconds

Parent: Writing data to shared memory of size 102400 bytes...
Child: Reading data from shared memory of size 102400 bytes...
Child: Writing data back to shared memory.
Parent: Reading data from shared memory.
Time taken for shared memory communication of size 102400 bytes: 0.000866 seconds

Parent: Writing data to shared memory of size 1048576 bytes...
Child: Reading data from shared memory of size 1048576 bytes...
Child: Writing data back to shared memory.
Parent: Reading data from shared memory.
Time taken for shared memory communication of size 1048576 bytes: 0.006991 seconds

Parent: Writing data to shared memory of size 10485760 bytes...
Child: Reading data from shared memory of size 10485760 byte

Part 2:

#  Message passing implementation
I added 2 codes for message passing implementation

CODE 1
  
  Pipe Buffer Size Limitation:
Code1 terminates without returnnig the time for 1MB.
Pipes have a default buffer size limit. If we attempt to write data that exceeds this size without reading it first, the write operation can block indefinitely which means the parent keeps trying to write to the pipe, but the buffer is full, so it waits for the child to read the data.
The typical pipe buffer size on many systems is 64KB. Since I am trying to send larger data sizes (1MB = 1048576 bytes and 10MB = 10485760 bytes), the pipe buffer might overflow, leading to a block.

In [ ]:
%%writefile message_passing.c
// Include necessary libraries for input/output, memory allocation, process management, and time measurement
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>     // For pipe, fork, read, write, close
#include <sys/types.h>  // For various data types
#include <sys/wait.h>   // For wait function
#include <time.h>       // For time measurement

// Define sizes for experimentation
#define SIZE_10KB 10240    // Define 10KB size
#define SIZE_100KB 102400  // Define 100KB size
#define SIZE_1MB 1048576   // Define 1MB size
#define SIZE_10MB 10485760 // Define 10MB size

// Function to fill an array with random integers
void fill_array(int *arr, int size) {
    // Fill the array by iterating over it and assigning random integers
    for (int i = 0; i < size / sizeof(int); i++) {
        arr[i] = rand() % 100;  // Assign random integer between 0 and 99
    }
}

int main() {
    int pipe_fd[2];    // File descriptors for the pipe (pipe_fd[0] for read, pipe_fd[1] for write)
    pid_t pid;         // Process ID for fork()

    // Array of sizes to experiment with
    int data_sizes[] = {SIZE_10KB, SIZE_100KB, SIZE_1MB, SIZE_10MB};
    // Number of data sizes in the array
    int num_sizes = sizeof(data_sizes) / sizeof(data_sizes[0]);

    // Loop through each data size for experimentation
    for (int j = 0; j < num_sizes; j++) {
        int data_size = data_sizes[j];  // Set the current data size

        // Time measurement start
        clock_t start, end;    // Variables to hold start and end times
        double time_spent;     // Variable to hold the time difference

        start = clock();       // Record the start time

        // Create pipe
        if (pipe(pipe_fd) == -1) {  // Check for error in pipe creation
            perror("Pipe failed");  // Print error if pipe creation fails
            exit(1);                // Exit the program on error
        }

        // Fork the process to create a parent and child
        pid = fork();   // Fork the current process into a parent and a child

        if (pid < 0) {  // Check if fork failed
            perror("Fork failed");  // Print error message if fork fails
            exit(1);    // Exit the program on error
        }

        if (pid == 0) {  // Child process
            close(pipe_fd[1]);  // Close write end of the pipe in the child process

            // Allocate memory to store the received data
            int *received_array = (int *)malloc(data_size);
            if (received_array == NULL) {  // Check if memory allocation failed
                perror("Malloc failed");   // Print error message
                exit(1);  // Exit on error
            }

            // Read data from the pipe
            printf("Child: Reading data from pipe of size %d bytes...\n", data_size);
            fflush(stdout);  // Ensure the output is flushed and visible
            read(pipe_fd[0], received_array, data_size);  // Read the data from the pipe

            // Send the array back to the parent through the pipe
            printf("Child: Sending data back to parent through pipe.\n");
            fflush(stdout);  // Ensure the output is flushed and visible
            write(pipe_fd[1], received_array, data_size);  // Write the data back to the parent

            close(pipe_fd[0]);   // Close the read end of the pipe
            free(received_array); // Free the allocated memory
            exit(0);  // Exit the child process
        } else {
            // Parent process
            close(pipe_fd[0]);  // Close the read end of the pipe in the parent process

            // Allocate memory to store the data
            int *array = (int *)malloc(data_size);
            if (array == NULL) {  // Check if memory allocation failed
                perror("Malloc failed");   // Print error message
                exit(1);  // Exit on error
            }

            // Fill the array with random integers
            fill_array(array, data_size);  // Call function to populate array with random data

            // Write the data to the pipe
            printf("Parent: Writing data to pipe of size %d bytes...\n", data_size);
            fflush(stdout);  // Ensure the output is flushed and visible
            write(pipe_fd[1], array, data_size);  // Write the array data to the pipe

            // Wait for the child process to complete
            wait(NULL);  // Parent waits for child to finish

            // Read the data back from the pipe
            printf("Parent: Receiving data back from child through pipe.\n");
            fflush(stdout);  // Ensure the output is flushed and visible
            read(pipe_fd[0], array, data_size);  // Read the data back from the child process

            close(pipe_fd[1]);  // Close the write end of the pipe
            free(array);  // Free the allocated memory
        }

        // Time measurement end
        end = clock();  // Record the end time
        time_spent = (double)(end - start) / CLOCKS_PER_SEC;  // Calculate the time taken
        // Print the time spent for message passing of the current data size
        printf("Time taken for message passing with size %d bytes: %f seconds\n", data_size, time_spent);
        fflush(stdout);  // Ensure the output is flushed and visible
    }

    return 0;  // End of the program
}


Writing message_passing.c


In [ ]:
!gcc message_passing.c -o message_passing
!./message_passing

Parent: Writing data to pipe of size 10240 bytes...
Child: Reading data from pipe of size 10240 bytes...
Child: Sending data back to parent through pipe.
Parent: Receiving data back from child through pipe.
Time taken for message passing with size 10240 bytes: 0.000333 seconds
Parent: Writing data to pipe of size 102400 bytes...
Child: Reading data from pipe of size 102400 bytes...
Child: Sending data back to parent through pipe.
Parent: Receiving data back from child through pipe.
Time taken for message passing with size 102400 bytes: 0.000919 seconds
Child: Reading data from pipe of size 1048576 bytes...
Parent: Writing data to pipe of size 1048576 bytes...
Child: Sending data back to parent through pipe.


CODE 2 Message Passing

The approach in CODE 2, the program should correctly process all four data sizes (10KB, 100KB, 1MB, and 10MB) without hanging, and print the time taken for each. Read and write data in smaller chunks. Instead of writing the entire array in one go, write and read smaller chunks repeatedly. This way, we can prevent exceeding the pipe's buffer size.

In [ ]:
%%writefile message_passing.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <time.h>

#define SIZE_10KB 10240    // 10KB size
#define SIZE_100KB 102400  // 100KB size
#define SIZE_1MB 1048576   // 1MB size
#define SIZE_10MB 10485760 // 10MB size

#define CHUNK_SIZE 4096  // Define a chunk size (4KB for splitting large data)

void fill_array(int *arr, int size) {
    for (int i = 0; i < size / sizeof(int); i++) {
        arr[i] = rand() % 100; // Fill array with random integers
    }
}

void write_in_chunks(int fd, int *data, int size) {
    int bytes_written = 0;
    while (bytes_written < size) {
        int chunk_size = (size - bytes_written > CHUNK_SIZE) ? CHUNK_SIZE : size - bytes_written;
        write(fd, (char*)data + bytes_written, chunk_size);
        bytes_written += chunk_size;
    }
}

void read_in_chunks(int fd, int *data, int size) {
    int bytes_read = 0;
    while (bytes_read < size) {
        int chunk_size = (size - bytes_read > CHUNK_SIZE) ? CHUNK_SIZE : size - bytes_read;
        read(fd, (char*)data + bytes_read, chunk_size);
        bytes_read += chunk_size;
    }
}

int main() {
    int pipe_fd[2];    // Array for the pipe's file descriptors
    pid_t pid;

    int data_sizes[] = {SIZE_10KB, SIZE_100KB, SIZE_1MB, SIZE_10MB};
    int num_sizes = sizeof(data_sizes) / sizeof(data_sizes[0]);

    for (int j = 0; j < num_sizes; j++) {
        int data_size = data_sizes[j];  // Set current data size for this iteration

        clock_t start, end;
        double time_spent;

        start = clock();  // Start measuring time

        if (pipe(pipe_fd) == -1) {  // Create a pipe
            perror("Pipe failed");
            exit(1);
        }

        pid = fork();  // Fork the process

        if (pid < 0) {
            perror("Fork failed");
            exit(1);
        }

        if (pid == 0) {
            // Child process
            close(pipe_fd[1]);  // Close write end

            int *received_array = (int *)malloc(data_size);
            if (received_array == NULL) {
                perror("Malloc failed");
                exit(1);
            }

            // Read data from the parent in chunks
            printf("Child: Reading data from pipe of size %d bytes...\n", data_size);
            fflush(stdout);
            read_in_chunks(pipe_fd[0], received_array, data_size);

            // Send the array back to the parent in chunks
            printf("Child: Sending data back to parent through pipe.\n");
            fflush(stdout);
            write_in_chunks(pipe_fd[1], received_array, data_size);

            close(pipe_fd[0]);  // Close read end
            free(received_array);
            exit(0);
        } else {
            // Parent process
            close(pipe_fd[0]);  // Close read end

            int *array = (int *)malloc(data_size);
            if (array == NULL) {
                perror("Malloc failed");
                exit(1);
            }

            fill_array(array, data_size);  // Fill array with random integers

            // Write the array to the child in chunks
            printf("Parent: Writing data to pipe of size %d bytes...\n", data_size);
            fflush(stdout);
            write_in_chunks(pipe_fd[1], array, data_size);

            wait(NULL);  // Wait for child to complete

            // Read the array back from the child in chunks
            printf("Parent: Receiving data back from child through pipe.\n");
            fflush(stdout);
            read_in_chunks(pipe_fd[1], array, data_size);

            close(pipe_fd[1]);  // Close write end
            free(array);
        }

        end = clock();  // End measuring time
        time_spent = (double)(end - start) / CLOCKS_PER_SEC;
        printf("Time taken for message passing with size %d bytes: %f seconds\n", data_size, time_spent);
        fflush(stdout);
    }

    return 0;
}


Overwriting message_passing.c


In [ ]:
!gcc message_passing.c -o message_passing
!./message_passing

Parent: Writing data to pipe of size 10240 bytes...
Child: Reading data from pipe of size 10240 bytes...
Child: Sending data back to parent through pipe.
Parent: Receiving data back from child through pipe.
Time taken for message passing with size 10240 bytes: 0.000302 seconds
Parent: Writing data to pipe of size 102400 bytes...
Child: Reading data from pipe of size 102400 bytes...
Child: Sending data back to parent through pipe.
Parent: Receiving data back from child through pipe.
Time taken for message passing with size 102400 bytes: 0.000975 seconds
Child: Reading data from pipe of size 1048576 bytes...
Parent: Writing data to pipe of size 1048576 bytes...
Child: Sending data back to parent through pipe.
Parent: Receiving data back from child through pipe.
Time taken for message passing with size 1048576 bytes: 0.007599 seconds
Child: Reading data from pipe of size 10485760 bytes...
Parent: Writing data to pipe of size 10485760 bytes...
Child: Sending data back to parent through pip

Compare the performance for different data sizes and message frequencies:

10KB (10240 bytes) in (0.000326 seconds,  shared memory), ( 0.000302 seconds, message passing )

100KB (102400 bytes) in (0.000866,	 seconds,  shared memory), ( 0.000975 seconds, message passing )

1MB (1048576 bytes)in (0.006991 seconds, shared memory), ( 0.007599 seconds, message passing )

10MB (10485760 bytes)in (0.067014 seconds,  shared memory), ( 0.071361 seconds, message passing )

Analysis:

1. Small Data Sizes (10KB, 100KB):
10KB: Both shared memory (0.000326s) and message passing (0.000302s) have very close performance. Message passing is slightly faster for this small data size, but the difference is minimal.
100KB: Here, shared memory becomes marginally faster (0.000866s) compared to message passing (0.000975s), though the difference is still small. This may be due to the lower overhead in directly accessing shared memory compared to the overhead involved in writing to and reading from a pipe.
2. Medium Data Size (1MB):
1MB: For this size, shared memory (0.006991s) shows better performance than message passing (0.007599s). The larger the data, the more efficient shared memory becomes because it avoids the overhead of transferring data via pipes, which must split the data into chunks.
3. Large Data Size (10MB):
10MB: For the largest data size, shared memory (0.067014s) outperforms message passing (0.071361s) even more significantly. As data size increases, the advantage of shared memory becomes clearer. The reason for this is that in shared memory, processes can access the same memory space directly, whereas message passing has to rely on copying data back and forth between processes.

Conclusion:

For smaller data sizes (10KB, 100KB), the difference between shared memory and message passing is minimal, with message passing sometimes being slightly faster.
For larger data sizes (1MB, 10MB), shared memory becomes significantly more efficient, with a clear performance advantage over message passing.

3. Explanation on the core part of implementation

A. Shared Memory Core Implementation:

Shared memory allows processes to directly access the same block of memory, reducing the need for data copying.

Steps in the implementation:

Shared Memory Creation:
A shared memory segment is created using shmget(), and a pointer is attached to it using shmat(). Both the parent and child access this shared memory segment. Example: shmid = shmget(IPC_PRIVATE, size, IPC_CREAT | 0666) creates a shared memory segment, and shm_ptr = shmat(shmid, NULL, 0) attaches it to the process's address space.

Forking: As with message passing, a fork() system call is used to create a child process. Both processes can access the shared memory segment through their pointers.

Parent Process:
The parent writes data directly to the shared memory using the pointer (memcpy(shm_ptr, data, size)). After the child has written data back, the parent reads the data from shared memory.

Child Process:
The child reads the data from the shared memory pointer and can modify it or send it back to the parent via shared memory.

Core Advantage:

Shared memory is more efficient for large data transfers because it avoids the need to copy data between processes, offering better performance for larger data sizes.

B. Message Passing (via Pipes) Core Implementation:

Pipes are used to create a unidirectional communication channel between a parent and child process.

Steps in the implementation:

Pipe Creation:
A pipe is created using the pipe() system call. It returns two file descriptors: one for reading and one for writing.
Example: pipe(pipe_fd) creates the pipe, with pipe_fd[0] for reading and pipe_fd[1] for writing.

Forking:
A fork() system call is used to create a child process. Both the parent and child share the pipe file descriptors.

Parent Process:
The parent closes the reading end of the pipe (pipe_fd[0]) and writes data into the pipe using write(pipe_fd[1], data, size).
After the child process sends data back, the parent reads it using read(pipe_fd[0], buffer, size).

Child Process:
The child closes the writing end of the pipe (pipe_fd[1]) and reads data sent by the parent using read(pipe_fd[0], buffer, size).
The child then sends the data back using write(pipe_fd[1], data, size).
Core Advantage: Pipes work well for small to medium data transfers and are simpler to implement. However, for large data transfers, pipes become less efficient due to the need to copy data between processes.





4. Discussion on performance comparison between different IPC mechanisms in terms of frequency and data size

 Shared memory performs better than message passing as the data size increases, particularly for large data transfers (1MB and above). This is because no data copying between processes is required; both processes directly access the same memory location.
Also, Lower overhead is involved because once the memory segment is created and attached, reading/writing to the memory happens in-place.

Frequency Impact:
The performance of shared memory improves as the frequency of data exchange increases because there’s no extra cost for system calls like read() and write() after memory is mapped.
For high-frequency, high-volume data transfers, shared memory is significantly faster.

2. Message Passing (via Pipes):

Pipes perform efficiently for smaller data sizes, up to 100KB, but become less efficient as the data size increases.
Pipes introduce higher overhead for large data transfers because they involve copying data back and forth between processes via the kernel buffer, which takes time. For data sizes like 10MB, message passing via pipes incurs more overhead due to the increased number of chunks that must be written/read between processes.

Frequency Impact:
When message frequency increases, frequent exchanges of data, pipes incur a performance penalty due to repeated system calls (read() and write()) and context switching between the parent and child processes.
As the frequency of messages increases, the overhead of copying data grows, leading to slower performance compared to shared memory.